# Tutorial: Ammonia Shelter-in-Place Analysis

This notebook is a production-grade, ready-to-run walkthrough for **protective action decision-making** around an ammonia release using `pyELDQM`.

It covers:
- scenario configuration (source, weather, AEGL thresholds, building type, timing)
- Gaussian dispersion simulation and AEGL zone extraction
- shelter-in-place vs. evacuation recommendation per zone
- summary reporting of sample-point vote counts
- interactive Folium map with shelter and evacuation zones displayed inline

## 1) Runtime Notes

This notebook supports two execution modes:
1. **Installed package mode** (`pip install pyeldqm`)
2. **Workspace mode** (running inside the project repository)

The import cell automatically detects and handles both.

In [ ]:
from __future__ import annotations
import sys
from math import radians, cos, sin, asin, sqrt
from pathlib import Path
from datetime import datetime
import numpy as np
import folium
from shapely.geometry import mapping as geom_mapping
from IPython.display import display, Markdown, HTML

IMPORT_MODE = "package"
try:
    from pyeldqm.core.dispersion_models.gaussian_model import calculate_gaussian_dispersion
    from pyeldqm.core.meteorology.stability import get_stability_class
    from pyeldqm.core.chemical_database import ChemicalDatabase
    from pyeldqm.core.utils.features import setup_computational_grid
    from pyeldqm.core.utils.zone_extraction import extract_zones
    from pyeldqm.core.protective_actions import analyze_shelter_zones
except ModuleNotFoundError:
    IMPORT_MODE = "workspace"
    cwd = Path.cwd().resolve()
    candidates = [cwd] + list(cwd.parents)
    pkg_dir = next((p / "pyeldqm" for p in candidates if (p / "pyeldqm").exists()), None)
    if pkg_dir is None:
        raise RuntimeError(
            "Could not resolve project root. Run from repository root or install pyeldqm via pip."
        )
    project_root = pkg_dir.parent
    if str(project_root) not in sys.path:
        sys.path.insert(0, str(project_root))

    from pyeldqm.core.dispersion_models.gaussian_model import calculate_gaussian_dispersion
    from pyeldqm.core.meteorology.stability import get_stability_class
    from pyeldqm.core.chemical_database import ChemicalDatabase
    from pyeldqm.core.utils.features import setup_computational_grid
    from pyeldqm.core.utils.zone_extraction import extract_zones
    from pyeldqm.core.protective_actions import analyze_shelter_zones

display(Markdown(f"**Import mode:** `{IMPORT_MODE}`"))


## 2) Scenario Configuration

Update these values to model your own scenario.

In [ ]:
# Chemical
CHEMICAL_NAME = "AMMONIA"
MOLECULAR_WEIGHT = 17.03              # g/mol
# Release location
SOURCE_LAT = 31.6911
SOURCE_LON = 74.0822
TIMEZONE_OFFSET_HRS = 5.0
# Release parameters
RELEASE_TYPE = "single"               # single | multi
SOURCE_TERM_MODE = "continuous"       # continuous | instantaneous
RELEASE_RATE = 800.0                  # g/s (continuous)
TANK_HEIGHT = 3.0                     # m
DURATION_MINUTES = 30.0               # min
MASS_RELEASED_KG = 500.0              # kg (instantaneous mode)
TERRAIN_ROUGHNESS = "URBAN"           # URBAN | RURAL
RECEPTOR_HEIGHT_M = 1.5               # m
# Weather
WEATHER_MODE = "manual"
WIND_SPEED = 2.5                      # m/s
WIND_DIRECTION = 90.0                 # degrees
TEMPERATURE_C = 25.0                  # C
HUMIDITY = 65.0                       # %
CLOUD_COVER = 30.0                    # %
# AEGL thresholds (ppm)
AEGL_THRESHOLDS = {
    "AEGL-1": 30.0,
    "AEGL-2": 160.0,
    "AEGL-3": 1100.0,
}
# Computational grid
X_MAX = 12000
Y_MAX = 4000
NX = 500
NY = 500
# Shelter analysis parameters
BUILDING_TYPE = "industrial"          # industrial | residential | office
SHELTERING_TIME_MIN = 60.0            # duration occupants shelter in place (min)
EVACUATION_TIME_MIN = 15.0            # time required to evacuate (min)
SAMPLE_GRID_POINTS = 15               # sample resolution for vote-based recommendation

assert NX > 10 and NY > 10, "Grid resolution is too low."
assert X_MAX > 0 and Y_MAX > 0, "Grid extents must be positive."
assert WIND_SPEED > 0, "Wind speed must be > 0."
assert SAMPLE_GRID_POINTS > 0, "SAMPLE_GRID_POINTS must be at least 1."
print("Scenario configured successfully.")
print(f"Building type        : {BUILDING_TYPE}")
print(f"Sheltering time      : {SHELTERING_TIME_MIN:.0f} min")
print(f"Evacuation time      : {EVACUATION_TIME_MIN:.0f} min")
print("All outputs will be shown inline in this notebook.")

## 3) Helper Functions

Distance utility for reporting maximum zone reach from the source.

In [ ]:
def haversine_km(lat1, lon1, lat2, lon2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    return 2 * asin(sqrt(a)) * 6371.0


def max_distance_from_source_km(polygon, src_lat, src_lon):
    if polygon is None or polygon.is_empty:
        return None
    max_d = 0.0
    try:
        for lon, lat in polygon.exterior.coords:
            d = haversine_km(src_lat, src_lon, lat, lon)
            if d > max_d:
                max_d = d
    except Exception:
        return None
    return max_d if max_d > 0 else None

## 4) Chemical Properties

Fetch core records from the chemical database for transparency and traceability.

In [ ]:
print("=" * 72)
print("CHEMICAL PROPERTIES")
print("=" * 72)
try:
    db = ChemicalDatabase()
    chem_data = next(
        (c for c in db.get_all_chemicals() if c.get("name") == CHEMICAL_NAME),
        None,
    )
    if chem_data:
        fields = [
            ("Name",                  "name"),
            ("Molecular Weight",      "molecular_weight"),
            ("Boiling Point (K)",     "boiling_point_K"),
            ("Density (kg/m3)",       "density_kgm3"),
            ("Vapor Pressure (kPa)",  "vapor_pressure_kPa"),
            ("IDLH (ppm)",            "idlh_ppm"),
            ("LC50 (ppm)",            "lc50_ppm"),
            ("AEGL-1 (60 min)",       "aegl1_60min"),
            ("AEGL-2 (60 min)",       "aegl2_60min"),
            ("AEGL-3 (60 min)",       "aegl3_60min"),
            ("ERPG-1",                "erpg1"),
            ("ERPG-2",                "erpg2"),
            ("ERPG-3",                "erpg3"),
        ]
        for label, key in fields:
            value = chem_data.get(key)
            if value not in (None, "", 0):
                print(f"{label:<26}: {value}")
    else:
        print(f"Name                     : {CHEMICAL_NAME}")
        print(f"Molecular Weight         : {MOLECULAR_WEIGHT} g/mol")
        print("Database record not found; using configured molecular weight.")
except Exception as ex:
    print(f"Name                     : {CHEMICAL_NAME}")
    print(f"Molecular Weight         : {MOLECULAR_WEIGHT} g/mol")
    print(f"Database query failed     : {ex}")

## 5) Run Dispersion and Extract AEGL Zones

In [ ]:
simulation_datetime = datetime.now()
weather = {
    "wind_speed":    WIND_SPEED,
    "wind_dir":      WIND_DIRECTION,
    "temperature_K": TEMPERATURE_C + 273.15,
    "humidity":      HUMIDITY / 100.0,
    "cloud_cover":   CLOUD_COVER / 100.0,
    "source":        "manual",
}
stability_class = get_stability_class(
    wind_speed=weather["wind_speed"],
    datetime_obj=simulation_datetime,
    latitude=SOURCE_LAT,
    longitude=SOURCE_LON,
    cloudiness_index=weather.get("cloud_cover", 0) * 10,
    timezone_offset_hrs=TIMEZONE_OFFSET_HRS,
)
X, Y, _, _ = setup_computational_grid(x_max=X_MAX, y_max=Y_MAX, nx=NX, ny=NY)
release_duration_s = DURATION_MINUTES * 60.0
source_q = RELEASE_RATE
dispersion_mode = "continuous" if SOURCE_TERM_MODE == "continuous" else "instantaneous"
concentration, U_local, resolved_stability, resolved_sources = calculate_gaussian_dispersion(
    weather=weather,
    X=X,
    Y=Y,
    source_lat=SOURCE_LAT,
    source_lon=SOURCE_LON,
    molecular_weight=MOLECULAR_WEIGHT,
    default_release_rate=source_q,
    default_height=TANK_HEIGHT,
    z_ref=3.0,
    z_measurement=RECEPTOR_HEIGHT_M,
    t=release_duration_s,
    t_r=release_duration_s,
    mode=dispersion_mode,
    sources=[{
        "lat": SOURCE_LAT, "lon": SOURCE_LON,
        "name": "Release Source",
        "height": TANK_HEIGHT, "rate": source_q,
        "color": "red",
    }],
    latitude=SOURCE_LAT,
    longitude=SOURCE_LON,
    timezone_offset_hrs=TIMEZONE_OFFSET_HRS,
    roughness=TERRAIN_ROUGHNESS,
    datetime_obj=simulation_datetime,
)
threat_zones = extract_zones(
    X, Y, concentration, AEGL_THRESHOLDS,
    SOURCE_LAT, SOURCE_LON, wind_dir=weather["wind_dir"],
)
print(f"Simulation time       : {simulation_datetime}")
print(f"Resolved wind speed   : {U_local:.2f} m/s")
print(f"Stability class       : {resolved_stability}")
print(f"Grid shape            : {concentration.shape}")

## 6) Report Zone Distances

Maximum distance is measured from source to polygon boundary.

In [ ]:
print("=" * 72)
print("METEOROLOGICAL CONDITIONS")
print("=" * 72)
print(f"Wind Speed            : {weather['wind_speed']:.2f} m/s")
print(f"Wind Direction        : {weather['wind_dir']:.0f} deg")
print(f"Temperature           : {weather['temperature_K'] - 273.15:.1f} C")
print(f"Humidity              : {weather['humidity'] * 100:.0f} %")
print(f"Cloud Cover           : {weather['cloud_cover'] * 100:.0f} %")
print(f"Stability Class       : {resolved_stability}")
print()
print("=" * 72)
print("AEGL THREAT ZONE DISTANCES FROM SOURCE")
print("=" * 72)
for zone_name in ["AEGL-3", "AEGL-2", "AEGL-1"]:
    poly = threat_zones.get(zone_name)
    threshold = AEGL_THRESHOLDS.get(zone_name, "N/A")
    threshold_str = f"{threshold:.0f} ppm" if isinstance(threshold, (int, float)) else str(threshold)
    dist_km = max_distance_from_source_km(poly, SOURCE_LAT, SOURCE_LON) if poly else None
    if dist_km is not None:
        print(
            f"{zone_name:<8} threshold: {threshold_str:<10} max distance: {dist_km:.3f} km ({dist_km * 1000:.0f} m)"
        )
    else:
        print(f"{zone_name:<8} threshold: {threshold_str:<10} max distance: -- (no zone formed)")

## 7) Run Shelter Analysis

`analyze_shelter_zones` samples `SAMPLE_GRID_POINTS × SAMPLE_GRID_POINTS` locations inside
each zone and votes **SHELTER IN PLACE** or **EVACUATE** based on the building type's
protection factor, sheltering duration, and evacuation time.

In [ ]:
shelter_result = analyze_shelter_zones(
    threat_zones=threat_zones,
    source_lat=SOURCE_LAT,
    source_lon=SOURCE_LON,
    building_type=BUILDING_TYPE,
    sheltering_time_minutes=SHELTERING_TIME_MIN,
    evacuation_time_minutes=EVACUATION_TIME_MIN,
    grid_points=SAMPLE_GRID_POINTS,
)

shelter_zones = []
evacuate_zones = []
total_sampled = 0
total_shelter = 0
total_evacuate = 0

for zone_name, zone_info in shelter_result.items():
    poly = threat_zones.get(zone_name)
    rec = zone_info.get("primary_recommendation", "SHELTER")
    total_sampled += zone_info.get("total_samples", 0)
    total_shelter += zone_info.get("shelter_count", 0)
    total_evacuate += zone_info.get("evacuate_count", 0)
    entry = {"name": zone_name, "polygon": poly, "recommendation": rec}
    if rec == "EVACUATE":
        evacuate_zones.append(entry)
    else:
        shelter_zones.append(entry)

overall_primary = "EVACUATE" if total_evacuate > total_shelter else "SHELTER IN PLACE"

print(f"Building type         : {BUILDING_TYPE}")
print(f"Sheltering time       : {SHELTERING_TIME_MIN:.0f} min")
print(f"Evacuation time       : {EVACUATION_TIME_MIN:.0f} min")
print(f"Sample points total   : {total_sampled}")
print(f"Shelter votes         : {total_shelter}")
print(f"Evacuate votes        : {total_evacuate}")
print(f"Overall recommendation: {overall_primary}")

## 8) Report Per-Zone Recommendations

In [ ]:
print("=" * 72)
print("SHELTER ANALYSIS — PER-ZONE RECOMMENDATIONS")
print("=" * 72)
print(f"{'Zone':<10} {'Recommendation':<20} {'Shelter':>8} {'Evacuate':>9} {'Sampled':>8}")
print("-" * 72)
for zone_name, zone_info in shelter_result.items():
    rec = zone_info.get("primary_recommendation", "SHELTER")
    n_shelter = zone_info.get("shelter_count", 0)
    n_evacuate = zone_info.get("evacuate_count", 0)
    n_total = zone_info.get("total_samples", 0)
    print(f"{zone_name:<10} {rec:<20} {n_shelter:>8} {n_evacuate:>9} {n_total:>8}")
print("-" * 72)
print(f"{'OVERALL':<10} {overall_primary:<20} {total_shelter:>8} {total_evacuate:>9} {total_sampled:>8}")
print("=" * 72)

## 9) Build Shelter Analysis Map (Inline)

Map layer colors:
- **Blue** — AEGL hazard zone boundary
- **Green** — zones where shelter-in-place is recommended
- **Red** — zones where evacuation is recommended

In [ ]:
m = folium.Map(location=[SOURCE_LAT, SOURCE_LON], zoom_start=13, tiles="OpenStreetMap")

# AEGL hazard zone outlines
for zone_name, zone_poly in threat_zones.items():
    if zone_poly is None or zone_poly.is_empty:
        continue
    folium.GeoJson(
        {"type": "Feature", "geometry": geom_mapping(zone_poly), "properties": {}},
        style_function=lambda _: {
            "fillColor": "#66a3ff", "color": "#2f5fb3",
            "weight": 1.5, "fillOpacity": 0.15,
        },
        tooltip=zone_name,
        name=zone_name,
    ).add_to(m)

# Shelter-in-place zones (green)
shelter_fg = folium.FeatureGroup(name="Shelter-in-Place Zones", show=True)
for zone in shelter_zones:
    poly = zone.get("polygon")
    if poly is None or poly.is_empty:
        continue
    folium.GeoJson(
        {"type": "Feature", "geometry": geom_mapping(poly), "properties": {}},
        style_function=lambda _: {
            "fillColor": "#90EE90", "color": "#228B22",
            "weight": 2, "fillOpacity": 0.35,
        },
        tooltip=f"{zone['name']}: SHELTER IN PLACE",
    ).add_to(shelter_fg)
shelter_fg.add_to(m)

# Evacuation zones (red)
evacuate_fg = folium.FeatureGroup(name="Evacuation Zones", show=True)
for zone in evacuate_zones:
    poly = zone.get("polygon")
    if poly is None or poly.is_empty:
        continue
    folium.GeoJson(
        {"type": "Feature", "geometry": geom_mapping(poly), "properties": {}},
        style_function=lambda _: {
            "fillColor": "#FF6347", "color": "#B22222",
            "weight": 2, "fillOpacity": 0.35,
        },
        tooltip=f"{zone['name']}: EVACUATE",
    ).add_to(evacuate_fg)
evacuate_fg.add_to(m)

# Release source marker
folium.Marker([SOURCE_LAT, SOURCE_LON], tooltip="Release Source").add_to(m)

folium.LayerControl().add_to(m)
print(f"Shelter analysis map generated. Overall recommendation: {overall_primary}")
print("Displaying inline below.")

## 10) Display Shelter Analysis Map Inline

In [ ]:
if 'm' in globals() and m is not None:
    display(HTML(m._repr_html_()))
else:
    print("Map object not found. Run Cell 19 first.")

## 11) Inline Results Summary

Quick run summary for tutorial users.

In [ ]:
peak_ppm = float(np.nanmax(concentration))
zone_stats = {}
for zone_name in ["AEGL-3", "AEGL-2", "AEGL-1"]:
    poly = threat_zones.get(zone_name)
    zone_stats[zone_name] = max_distance_from_source_km(poly, SOURCE_LAT, SOURCE_LON) if poly else None
print("Completed successfully.")
print(f"Chemical              : {CHEMICAL_NAME}")
print(f"Peak concentration    : {peak_ppm:.2f} ppm")
print(f"Building type         : {BUILDING_TYPE}")
print(f"Overall recommendation: {overall_primary}")
print(f"Shelter-in-place zones: {len(shelter_zones)}")
print(f"Evacuation zones      : {len(evacuate_zones)}")
print(f"Sample points total   : {total_sampled}")
print("Max zone distances:")
for z, d in zone_stats.items():
    if d is None:
        print(f"  {z}: no zone formed")
    else:
        print(f"  {z}: {d:.3f} km ({d * 1000:.0f} m)")

## Troubleshooting

- If imports fail, run from repository root **or** install package mode: `pip install pyeldqm`.

- If the map does not render inline, rerun Cell 19 then Cell 21.

- If zones are empty, increase release rate/duration, reduce thresholds, or expand the grid extent.

- If all zones return the same recommendation, adjust `SHELTERING_TIME_MIN`, `EVACUATION_TIME_MIN`,
  or `BUILDING_TYPE` to better reflect your scenario conditions.

- Increase `SAMPLE_GRID_POINTS` for finer spatial resolution in the vote-based decision (at the
  cost of slightly longer execution time).